In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

# Stocks History 
Stocks_Symbol = 'SPY'
df_Stocks = yf.download(Stocks_Symbol, start='2022-01-01', end=pd.Timestamp.today().strftime('%Y-%m-%d'))[['Close']]
df_Stocks.columns = ['Close']

# Options History 
Options = yf.Ticker(Stocks_Symbol)
Options_Expiration = Options.options
Options_Chain = Options.option_chain(Options_Expiration[1])  # Second soonest expiry
Options_Calls = Options_Chain.calls.copy()
Options_Puts = Options_Chain.puts.copy()
Options_Calls['Option_Type'] = "Call"
Options_Puts['Option_Type'] = "Put"
Options_Calls['Expiration'] = pd.to_datetime("20" + Options_Calls['contractSymbol'].str.extract(r'(\d{6})')[0], format='%Y%m%d')
Options_Puts['Expiration'] = pd.to_datetime("20" + Options_Puts['contractSymbol'].str.extract(r'(\d{6})')[0], format='%Y%m%d')
Options_Calls = Options_Calls[Options_Calls['impliedVolatility'] > 0.01].reset_index(drop=True)
Options_Puts = Options_Puts[Options_Puts['impliedVolatility'] > 0.01].reset_index(drop=True)

# Options Chain Based On ATM Option Price 
Options_ATM_Calls = Options_Calls.iloc[(Options_Calls['strike'] - df_Stocks['Close'].iloc[-1]).abs().argmin()]
Options_ATM_Puts = Options_Puts.iloc[(Options_Puts['strike'] - df_Stocks['Close'].iloc[-1]).abs().argmin()]

# Historical Volatility and Annual Drift
Historical_Volatility_Window = 30
Annual_Drift_Window = 252
Risk_Free_Rate = 0.05
df_Stocks['Historical_Volatility'] = ((np.log(df_Stocks['Close'] / df_Stocks['Close'].shift(1)).rolling(Historical_Volatility_Window).std()) * np.sqrt(252)).fillna(0)
df_Stocks['Annual_Drift'] = (((np.log(df_Stocks['Close'] / df_Stocks['Close'].shift(1))).rolling(Annual_Drift_Window).mean()) * 252).fillna(0)

# Black-Scholes Setup
Black_Scholes_T = ((Options_ATM_Calls['Expiration'] - pd.Timestamp.today().normalize()).days) / 252
Current_Stock_Price = df_Stocks['Close'].iloc[-1]

# Black-Scholes Functions (for Delta and Vega only)
def Black_Scholes_Calls_Price(Sigma, Current_Stock_Price, Return_Delta_And_Price=False):
    D1 = (np.log(Current_Stock_Price / Options_ATM_Calls['strike']) + (Risk_Free_Rate + 0.5 * (Sigma ** 2)) * Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    Price = Current_Stock_Price * norm.cdf(D1) - (Options_ATM_Calls['strike'] * np.exp(-Risk_Free_Rate * Black_Scholes_T) * norm.cdf(D1 - Sigma * np.sqrt(Black_Scholes_T)))
    Delta = norm.cdf(D1)
    if Return_Delta_And_Price:
        return Price, Delta
    return Price

def Black_Scholes_Puts_Price(Sigma, Current_Stock_Price, Return_Delta_And_Price=False):
    D1 = (np.log(Current_Stock_Price / Options_ATM_Puts['strike']) + (Risk_Free_Rate + 0.5 * (Sigma ** 2)) * Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    Price = (Options_ATM_Puts['strike'] * np.exp(-Risk_Free_Rate * Black_Scholes_T) * norm.cdf(-(D1 - Sigma * np.sqrt(Black_Scholes_T)))) - Current_Stock_Price * norm.cdf(-D1)
    Delta = norm.cdf(D1)
    if Return_Delta_And_Price:
        return Price, Delta
    return Price

def Vega_Calls(Sigma):
    D1 = (np.log(Current_Stock_Price / Options_ATM_Calls['strike']) + (Risk_Free_Rate + 0.5 * (Sigma ** 2)) * Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    return Current_Stock_Price * np.sqrt(Black_Scholes_T) * norm.pdf(D1)

def Vega_Puts(Sigma):
    D1 = (np.log(Current_Stock_Price / Options_ATM_Puts['strike']) + (Risk_Free_Rate + 0.5 * (Sigma ** 2)) * Black_Scholes_T) / (Sigma * np.sqrt(Black_Scholes_T))
    return Current_Stock_Price * np.sqrt(Black_Scholes_T) * norm.pdf(D1)

# Newton-Raphson Implied Volatility 
Sigma_Calls = df_Stocks['Historical_Volatility'].iloc[-1]
Sigma_Puts = df_Stocks['Historical_Volatility'].iloc[-1]

for i in range(100):
    Sigma_Delta = Black_Scholes_Calls_Price(Sigma_Calls, Current_Stock_Price) - Options_ATM_Calls['lastPrice']
    Vega_Delta = Vega_Calls(Sigma_Calls)
    if abs(Sigma_Delta) < 0.0001:
        break
    Sigma_Calls = Sigma_Calls - Sigma_Delta / Vega_Delta
Newton_Raphson_Implied_Volatility_Calls = Sigma_Calls

for i in range(100):
    Sigma_Delta = Black_Scholes_Puts_Price(Sigma_Puts, Current_Stock_Price) - Options_ATM_Puts['lastPrice']
    Vega_Delta = Vega_Puts(Sigma_Puts)
    if abs(Sigma_Delta) < 0.0001:
        break
    Sigma_Puts = Sigma_Puts - Sigma_Delta / Vega_Delta
Newton_Raphson_Implied_Volatility_Puts = Sigma_Puts

# Delta Neutral Trading Strategy 
Strategy_HV = df_Stocks['Historical_Volatility'].iloc[-1]
Strategy_IV_Calls = Newton_Raphson_Implied_Volatility_Calls
Strategy_IV_Puts = Newton_Raphson_Implied_Volatility_Puts
Strategy_Entry = 0.1
Strategy_TP = 0.02
Strategy_SL = 0.3
Contract_Size = 100

Strategy_IV_Diff_Calls = Strategy_HV - Strategy_IV_Calls
Strategy_IV_Diff_Puts = Strategy_HV - Strategy_IV_Puts

# Stock Delta Neutral Hedge
def Delta_Neutral_Hedge(Delta_Hedge, Contracts=1):
    Stock_Shares = -Delta_Hedge * Contracts * Contract_Size
    return Stock_Shares

# Prints : Basis
print(f"{Stocks_Symbol} -> Closing Price : {df_Stocks['Close'].iloc[-1]:.2f} | Annual Drift : {(df_Stocks['Annual_Drift'].iloc[-1]) * 100:.2f}% | Historical_Volatility : {(df_Stocks['Historical_Volatility'].iloc[-1]*100):.2f}%")
print(f"Call Strike Price : {Options_ATM_Calls['strike']} | Implied Volatility : {Newton_Raphson_Implied_Volatility_Calls*100:.2f}% (Newton-Raphson) | Expiration Date : {Options_ATM_Calls['Expiration'].strftime('%Y-%m-%d')}")
print(f"Put Strike Price : {Options_ATM_Puts['strike']} | Implied Volatility : {Newton_Raphson_Implied_Volatility_Puts*100:.2f}% (Newton-Raphson) | Expiration Date : {Options_ATM_Puts['Expiration'].strftime('%Y-%m-%d')}")

# Volatility Arbitrage
Strategy_Dictionary = {'Call': {}, 'Put': {}}
print("\nDelta Neutral Volatility Arbitrage:")
for Option_Type, Volatility_Diff, Strategy_IV, Option_Data, BS_Func in [
    ('Call', Strategy_IV_Diff_Calls, Strategy_IV_Calls, Options_ATM_Calls, Black_Scholes_Calls_Price),
    ('Put', Strategy_IV_Diff_Puts, Strategy_IV_Puts, Options_ATM_Puts, Black_Scholes_Puts_Price)
]:
    _, Delta = BS_Func(Strategy_IV, Current_Stock_Price, Return_Delta_And_Price=True)
    Stock_Shares_Hedge = Delta_Neutral_Hedge(Delta)
    
    # Calculate TP and SL prices
    entry_price = Option_Data['lastPrice']
    if abs(Volatility_Diff) > Strategy_Entry:
        # Calculate TP based on HV +/- 2%
        if Volatility_Diff > 0:  # Buy (Long) - TP when HV increases by 2%
            tp_vol = Strategy_HV + Strategy_TP
            tp_price = BS_Func(tp_vol, Current_Stock_Price)
            sl_price = entry_price * (1 - Strategy_SL)  # SL at 70% of entry
        else:  # Sell (Short) - TP when HV decreases by 2%
            tp_vol = Strategy_HV - Strategy_TP
            tp_price = BS_Func(tp_vol, Current_Stock_Price)
            sl_price = entry_price * (1 + Strategy_SL)  # SL at 130% of entry

        if Volatility_Diff > 0:  # HV > IV (Option underpriced, buy it)
            Action = "Buy"
            Position = 1
            print(f"{Option_Type} Entry: HV ({Strategy_HV*100:.2f}%) > IV ({Strategy_IV*100:.2f}%), Difference: {Volatility_Diff*100:.2f}%")
            print(f"  Buy 1 {Option_Type} contract @ {entry_price * Contract_Size:.2f}, Delta: {Delta:.4f}")
            print(f"  Hedge: Sell {abs(Stock_Shares_Hedge):.2f} stock shares")
            print(f"  TP: {tp_price * Contract_Size:.2f} (at HV {tp_vol*100:.2f}%) | SL: {sl_price * Contract_Size:.2f}")
        else:  # IV > HV (Option overpriced, sell it)
            Action = "Sell"
            Position = -1
            print(f"{Option_Type} Entry: IV ({Strategy_IV*100:.2f}%) > HV ({Strategy_HV*100:.2f}%), Difference: {-Volatility_Diff*100:.2f}%")
            print(f"  Sell 1 {Option_Type} contract @ {entry_price * Contract_Size:.2f}, Delta: {Delta:.4f}")
            print(f"  Hedge: Buy {abs(Stock_Shares_Hedge):.2f} stock shares")
            print(f"  TP: {tp_price * Contract_Size:.2f} (at HV {tp_vol*100:.2f}%) | SL: {sl_price * Contract_Size:.2f}")
        
        Strategy_Dictionary[Option_Type] = {
            'Action': Action,
            'Position': Position,
            'Entry_Price': entry_price,
            'Strike': Option_Data['strike'],
            'Expiration': Option_Data['Expiration'],
            'Shares': Stock_Shares_Hedge,
            'Entry_Diff': Volatility_Diff,
            'TP_Price': tp_price,
            'SL_Price': sl_price,
            'TP_Vol': tp_vol,
            'Active': True
        }
    else:
        print(f"{Option_Type}: No entry - Difference {abs(Volatility_Diff)*100:.2f}% < {Strategy_Entry*100}%")
        Strategy_Dictionary[Option_Type]['Active'] = False

[*********************100%***********************]  1 of 1 completed

SPY -> Closing Price : 567.13 | Annual Drift : 11.62% | Historical_Volatility : 17.87%
Call Strike Price : 569.0 | Implied Volatility : 21.64% (Newton-Raphson) | Expiration Date : 2025-03-21
Put Strike Price : 565.0 | Implied Volatility : 29.36% (Newton-Raphson) | Expiration Date : 2025-03-21

Delta Neutral Volatility Arbitrage:
Call: No entry - Difference 3.77% < 10.0%
Put Entry: IV (29.36%) > HV (17.87%), Difference: 11.48%
  Sell 1 Put contract @ 315.00, Delta: 0.5884
  Hedge: Buy 58.84 stock shares
  TP: 131.17 (at HV 15.87%) | SL: 409.50
